# Automated Tesing & Continuous Integration

When writing software, especially scientific software, a key question is whether the code is correct, and provably correct. How do we check correctness? We test.

Lets have a concrete example. We'll look for solutions to the quadratic equation
$$ ax^2 + bx + c =0,$$
where $x$ is a real number. The formula for solutions is
$$ x=\frac{-b \pm \sqrt{b^2-4ac}}{2a}$$.
Now to write some code

In [1]:
import numpy as np # for the sqrt function

def solve_quadratic(a,b,c):
    """Solve a quadratic equation ax^2+bx+c=0 in the reals"""
    if 0<b**2-4.0*a*c:
        # two solutions
        return ((-b-np.sqrt(b**2-4.0*a*c))/(2.0*a),(-b+np.sqrt(b**2-4.0*a*c))/(2.0*a))
    elif 0==b**2-4.0*a*c:
        # one solution
        return -b/(2.0*a)
    else:
        # no solutions
        return None

We can try some ad hoc tests:

In [2]:
# solve x^2-1=0
solve_quadratic(1,0,-1)==(-1.0,1.0)

True

In [3]:
solve_quadratic(1,0,0)==(0.0)

True

In [4]:
solve_quadratic(1,0,1) is None

True

Humans are lazy, so we want to make running tests when the code changes as easy as possible. We roll the tests into a single function, and use the `assert()` function so that it throws an exception if anything goes wrong.

In [5]:
import foo

def test_solve_quadratic():
    
    assert(foo.solve_quadratic(1,0,-1) == (-1.0,1.0))
    assert(foo.solve_quadratic(1,0,0) == (0.0))
    assert(foo.solve_quadratic(1,0,1) is None)
    
test_solve_quadratic()

In [6]:
test_solve_quadratic()

# The `unittest` module

It's still useful to automate things a bit more, so that we can . Python provides an inbuilt `unittest` module, which (with some work) can be used to build a test framework. It introduces the concept of the three stages of a test:
1. Set up. We create anything which must already exist for a test to make sense.
2. Running. The test itself operates
3. Tear down. We clean up anything which won't get dealt with automatically

The file `unittest_example.py` contains the following code

```python
import unittest
import foo

class TestSolveQuadratic(unittest.TestCase):
    def test_solve_quadratic(self):
    
        self.assertEqual(foo.solve_quadratic(1,0,-1), (-1.0,1.0))
        self.assertEqual(foo.solve_quadratic(1,0,0), 0.0 )
        self.assertEqual(foo.solve_quadratic(1,0,1), None)
   
   
unittest.main()
```
        

In [7]:
!python3 unittest_example.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


The external `pytest` functions simplifies this even further, as well as providing a more informative interface. The file `pytest_example.py` contains the following code

```python
import foo

def test_solve_quadratic():
    
    assert(foo.solve_quadratic(1,0,-1)==(-1.0,1.0))
    assert(foo.solve_quadratic(1,0,0)==0.0)
    assert(foo.solve_quadratic(1,0,1) is None)
    
```

In [8]:
!py.test-3.6 pytest_example.py

============================= test session starts ==============================
platform darwin -- Python 3.6.5, pytest-3.4.0, py-1.5.2, pluggy-0.6.0
rootdir: /Users/origimbo/MscASCE/teaching, inifile:
plugins: nbval-0.9.1
collected 1 item                                                               

pytest_example.py .                                                      [100%]

=========================== 1 passed in 0.18 seconds ===========================


## Continuous integration

Once you have tests, you want to make sure that any public changes to the code don't break them. For code under version control, this means that you want to rerun your test suite on all new public merges, to ensure that the code installs and runs successfully on a fresh system. Continous integration (CI) frameworks tie together version control repositories and test suites so that this happens automatically.

As with test frameworks, a large number of CI solutions exist. We will introduce a web-based framework called Travis, which interacts nicely with GitHub.

Travis is configured from a file called `.travis.yml` in the root directory of the repository. In this case, the file looks like

```yml
language: python
python:
  - "2.7"
  - "3.6"
# command to install dependencies
install:
  - pip install -r requirements.txt
# command to run tests on script and regression tests on notebook
script: pytest pytest_example.py
```

Here the first line specifies that we're using python (and thus that travis needs to create a virtual machine running python. The next lines say we want to test python 2.7 as well as python3.6. The `install:` command tells Travis to install extra python packages our code requires using the `pip` package manager. For this repository, our `requirements.txt` file contains only one line:

```txt
numpy>=13.0
```

This says that we need a version of `numpy` at 13.0 or more recent.